In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import xopr

import holoviews as hv
import xarray as xr
import hvplot
import hvplot.xarray
import hvplot.pandas
import geoviews.feature as gf
import cartopy.crs as ccrs
import rioxarray
from tqdm import tqdm

In [ ]:
opr = xopr.OPRConnection(cache_dir='radar_cache')

In [ ]:
epsg_3031 = ccrs.Stereographic(central_latitude=-90, true_scale_latitude=-71)
coastline = gf.coastline.options(scale='50m').opts(projection=epsg_3031)
velocity = rioxarray.open_rasterio(
    "https://its-live-data.s3.amazonaws.com/velocity_mosaic/v2/static/cog/ITS_LIVE_velocity_120m_RGI19A_0000_v02_v.tif",
    chunks='auto', overview_level=4, cache=False
).squeeze().drop_vars(['spatial_ref', 'band']).rename('velocity (m/year)')
velocity_map = velocity.hvplot.image(x='x', y='y', cmap='gray_r').opts(clim=(0,1000))

In [ ]:
region = xopr.geometry.get_antarctic_regions(name=["Vincennes_Bay", "Underwood"], merge_regions=True, simplify_tolerance=100)
region_projected = xopr.geometry.project_geojson(region, source_crs='EPSG:4326', target_crs="EPSG:3031")

region_hv = hv.Polygons([region_projected]).opts(
    color='green',
    line_color='black',
    fill_alpha=0.3)

(velocity_map * coastline * region_hv).opts(aspect='equal')

In [ ]:
gdf = opr.query_frames(geometry=region).to_crs('EPSG:3031')
print(f"Found {len(gdf)} radar frames in the selected region.")
gdf.head()

In [ ]:
radar_frames_hv = gdf.hvplot(by='collection', hover_cols=['id'])
(velocity_map * coastline * region_hv * radar_frames_hv).opts(aspect='equal', legend_position='top_left')

In [ ]:
layer_ds_list = []

with tqdm(gdf.iterrows(), total=len(gdf)) as t:
    for id, frame in t:
        t.set_description(f"{id}")
        layers = opr.get_layers(frame)
        bed_layer_name = None
        if 'standard:bottom' in layers:
            bed_layer_name = 'standard:bottom'
        elif ':bottom' in layers:
            bed_layer_name = ':bottom'
        else:
            continue  # No bed layer found
        layer_wgs84 = xopr.radar_util.layer_twtt_to_range(layers[bed_layer_name], layers["standard:surface"], vertical_coordinate='wgs84').rename({'lat': 'Latitude', 'lon': 'Longitude'})
        layer_wgs84 = xopr.geometry.project_dataset(layer_wgs84, target_crs='EPSG:3031')
        layer_wgs84 = layer_wgs84.dropna('slow_time', subset=['wgs84'])
        layer_wgs84['source'] = id
        layer_ds_list.append(layer_wgs84)

In [ ]:
bed_merged = xr.concat(layer_ds_list, dim='slow_time')

In [ ]:
bed_hv = bed_merged.hvplot.scatter(x='x', y='y', c='wgs84', cmap='turbo', s=2).opts(clabel='Bed Elevation WGS84 (m)')
(velocity_map.opts(colorbar=False) * coastline * region_hv * radar_frames_hv * bed_hv).opts(aspect='equal', legend_position='top_left')

In [ ]:
bed_merged

In [ ]:
import numpy as np
import verde as vd

# Get the extent of bed_merged data
x_min = bed_merged['x'].min().values
x_max = bed_merged['x'].max().values
y_min = bed_merged['y'].min().values
y_max = bed_merged['y'].max().values

print(f"Data extent: x=[{x_min:.0f}, {x_max:.0f}], y=[{y_min:.0f}, {y_max:.0f}]")

# Create 10 km (10000 m) grid spacing
spacing = 10_000

# Create grid coordinates using Verde's grid_coordinates
grid_coords = vd.grid_coordinates(
    region=(x_min, x_max, y_min, y_max),
    spacing=spacing,
    pixel_register=True  # Grid nodes at pixel centers
)

print(f"Grid shape: {grid_coords[0].shape}")
print(f"Number of grid cells: {grid_coords[0].size}")

In [ ]:
# Extract coordinates and wgs84 values
x_data = bed_merged['x'].values
y_data = bed_merged['y'].values
wgs84_data = bed_merged['wgs84'].values

# Use Verde's BlockMean to calculate mean in each 10km x 10km block
# BlockMean is a reducer that bins data and computes the mean
gridder = vd.BlockMean(spacing=spacing, region=(x_min, x_max, y_min, y_max))
filtered = gridder.filter(
    coordinates=(x_data, y_data), 
    data=wgs84_data
)

# Unpack the filtered results
block_coords = filtered[0]  # tuple of (x, y) arrays
block_means = filtered[1]   # array of mean values

print(f"Number of data points before reduction: {len(x_data)}")
print(f"Number of blocks with data: {len(block_means)}")

# Create grid coordinates for the full region
grid_x, grid_y = vd.grid_coordinates(
    region=(x_min, x_max, y_min, y_max),
    spacing=spacing,
    pixel_register=True
)

# Create an empty grid filled with NaN
grid_shape = grid_x.shape
bed_elevation_grid = np.full(grid_shape, np.nan)

# Fill in the grid with block means at their corresponding positions
# For each block with data, find its grid position and fill it
for i in range(len(block_means)):
    bx, by = block_coords[0][i], block_coords[1][i]
    # Find closest grid indices
    ix = np.argmin(np.abs(grid_x[0, :] - bx))
    iy = np.argmin(np.abs(grid_y[:, 0] - by))
    bed_elevation_grid[iy, ix] = block_means[i]

# Create xarray Dataset
bed_grid = xr.Dataset(
    data_vars={'bed_elevation': (['northing', 'easting'], bed_elevation_grid)},
    coords={'easting': grid_x[0, :], 'northing': grid_y[:, 0]}
)

bed_grid

In [ ]:
# Plot the gridded bed elevation data
bed_grid_plot = bed_grid['bed_elevation'].hvplot.image(
    x='easting', 
    y='northing', 
    cmap='turbo',
    clabel='Mean Bed Elevation WGS84 (m)',
    title='10 km Gridded Bed Elevation'
).opts(colorbar=True)

# Overlay with the original scatter points and other features
(velocity_map.opts(colorbar=False) * 
 coastline * 
 region_hv * 
 bed_grid_plot).opts(aspect='equal', legend_position='top_left')